In [65]:
import json
from datetime import datetime
from pathlib import Path

import albumentations as A
import cv2
import numpy as np
import polars as pl
import pytz
import timm
import torch
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from omegaconf import OmegaConf
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from transformers import get_cosine_schedule_with_warmup

from src.config import cfg
from src.dir import create_dir
from src.seed import seed_everything

cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)
pl.Config.set_fmt_str_lengths(1000)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


exp_number: '009'
run_time: base
data:
  input_root: ../../data/input
  train_path: ../../data/input/train_features.csv
  test_path: ../../data/input/test_features.csv
  sample_submission_path: ../../data/input/sample_submission.csv
  img_root: ../../data/input/images
  json_root: ../../data/input/traffic_lights
  depth_root: ../../data/input/depth
  output_root: ../../data/output
  results_root: ../../results
  results_path: ../../results/009/base
seed: 319
n_splits: 4
target_cols:
- x_0
- y_0
- z_0
- x_1
- y_1
- z_1
- x_2
- y_2
- z_2
- x_3
- y_3
- z_3
- x_4
- y_4
- z_4
- x_5
- y_5
- z_5
cnn:
  model_name: convnext_base.fb_in22k_ft_in1k
  size: 128
  pretrained: true
  in_chans: 20
  target_size: 18
  lr: 0.001
  num_epochs: 20
  batch_size: 32

Using device: cpu


### exp009

- late sub
- 信号機のマスク情報とdepth画像をチャネル方向に結合して入力とするNN
- 数値特徴量も入力とする
- backbone：convnext_base

### データの読み込み

In [72]:
# データの読み込み
train = pl.read_csv(cfg.data.train_path, try_parse_dates=True).with_row_index()
test = pl.read_csv(cfg.data.test_path, try_parse_dates=True).with_row_index()
sample_submission = pl.read_csv(cfg.data.sample_submission_path, try_parse_dates=True).with_row_index()


# scene列を作成 → これでGroupKFoldする
train = train.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))
test = test.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))

# データの結合(label encoding用)
train_test = pl.concat([train, test], how="diagonal")

# CV
gkf = GroupKFold(n_splits=cfg.n_splits)


In [74]:
# 数値特徴量の前処理
numeric_cols = ["vEgo", "aEgo", "steeringAngleDeg", "steeringTorque", "gas"]

# 欠損を埋める
train_test = train_test.with_columns([pl.col(col).fill_null(0) for col in numeric_cols])

# 全ての数値特徴量を標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(train_test[numeric_cols].to_numpy())

# 標準化した値でDataFrameを更新
train_test = train_test.with_columns(
    [pl.Series(scaled_features[:, i]).alias(col) for i, col in enumerate(numeric_cols)]
)


In [77]:
# train, testに分ける
train = train_test.filter(pl.col("x_0").is_not_null()).sort("index")
test = train_test.filter(pl.col("x_0").is_null()).sort("index")


### データ拡張


In [68]:
def get_train_transform():
    return A.ReplayCompose(
        [
            A.Resize(cfg.cnn.size, cfg.cnn.size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


def get_valid_transform():
    return A.ReplayCompose(
        [
            A.Resize(cfg.cnn.size, cfg.cnn.size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


### 信号機マスク

In [69]:
class TrafficLightMaskGenerator:
    TRAFFIC_LIGHT_CLASSES = ["green", "yellow", "red", "straight", "left", "right", "empty", "other"]

    def __init__(self, image_size: list[int, int] | int | None = None):
        self.original_size = (128, 64)  # (width, height)

        if isinstance(image_size, list):
            self.image_size = (image_size[1], image_size[0])
        elif isinstance(image_size, int):
            self.image_size = (image_size, image_size)
        else:
            self.image_size = self.original_size

        self.scale = (self.image_size[0] / self.original_size[0], self.image_size[1] / self.original_size[1])

        self.class_to_index = {cls: idx for idx, cls in enumerate(self.TRAFFIC_LIGHT_CLASSES)}

    def scale_bbox(self, bbox):
        x1, y1, x2, y2 = bbox
        return [x1 * self.scale[0], y1 * self.scale[1], x2 * self.scale[0], y2 * self.scale[1]]

    def _convert_coordinate(self, coord, max_size):
        """座標を適切な範囲に収める"""
        return max(0, min(int(round(coord)), max_size - 1))

    def generate_masks(self, traffic_lights_json):
        # PosixPathオブジェクトを文字列に変換
        if isinstance(traffic_lights_json, Path):
            traffic_lights_json = str(traffic_lights_json)

        traffic_lights = (
            json.load(open(traffic_lights_json)) if isinstance(traffic_lights_json, str) else traffic_lights_json
        )

        masks = np.zeros((self.image_size[1], self.image_size[0], len(self.TRAFFIC_LIGHT_CLASSES)), dtype=np.float32)

        for light in traffic_lights:
            class_idx = self.class_to_index[light["class"]]
            scaled_bbox = self.scale_bbox(light["bbox"])

            # 座標を適切な範囲に収める
            x1 = self._convert_coordinate(scaled_bbox[0], self.image_size[0])
            y1 = self._convert_coordinate(scaled_bbox[1], self.image_size[1])
            x2 = self._convert_coordinate(scaled_bbox[2], self.image_size[0])
            y2 = self._convert_coordinate(scaled_bbox[3], self.image_size[1])

            masks[y1 : y2 + 1, x1 : x2 + 1, class_idx] = 1.0

        return masks


### データセット

In [70]:
feature_cols = [
    "vEgo",
    "aEgo",
    "steeringAngleDeg",
    "steeringTorque",
    "gas",
    "leftBlinker",
    "rightBlinker",
    "brakePressed",
]


In [71]:
class CustomDataset(Dataset):
    def __init__(self, df, img_dir, json_dir, depth_dir, transform=None, is_train=True):
        self.df = df
        self.img_dir = Path(img_dir)
        self.json_dir = Path(json_dir)
        self.depth_dir = Path(depth_dir)
        self.is_train = is_train
        self.feature_cols = [
            "vEgo",
            "aEgo",
            "steeringAngleDeg",
            "steeringTorque",
            "gas",
            "leftBlinker",
            "rightBlinker",
            "brakePressed",
        ]

        # オリジナル画像用の変換処理
        if transform is None:
            self.transform = A.ReplayCompose(
                [
                    A.Resize(cfg.cnn.size, cfg.cnn.size),
                    A.Normalize(
                        mean=[0.485, 0.456, 0.406],  # 通常のImageNet平均値
                        std=[0.229, 0.224, 0.225],  # 通常のImageNet標準偏差
                    ),
                    ToTensorV2(),
                ]
            )
        else:
            self.transform = transform

        # Depth Map用の変換処理
        self.depth_transform = A.ReplayCompose(
            [
                A.Resize(cfg.cnn.size, cfg.cnn.size),
                A.Normalize(mean=[0.5], std=[0.5]),
                ToTensorV2(),
            ]
        )

        self.target_cols = cfg.target_cols
        self.mask_generator = TrafficLightMaskGenerator(cfg.cnn.size)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df[idx]
        img_folder = self.img_dir / row["ID"].item()
        depth_folder = self.depth_dir / row["ID"].item()
        json_path = self.json_dir / f"{row['ID'].item()}.json"

        # 3枚の画像を読み込み
        img_names = ["image_t-1.0.png", "image_t-0.5.png", "image_t.png"]
        imgs = []
        depths = []

        for img_name in img_names:
            # オリジナル画像の読み込み
            img_path = img_folder / img_name
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img)

            # Depth Mapの読み込み
            depth_path = depth_folder / img_name
            depth = cv2.imread(str(depth_path), cv2.IMREAD_GRAYSCALE)
            depths.append(depth)

        # オリジナル画像の変換
        if self.transform:
            replay = None
            transformed_imgs = []

            for img in imgs:
                if replay is None:
                    # 最初の画像に対して変換を実行し、replayを保存
                    transformed = self.transform(image=img)
                    replay = transformed["replay"]
                else:
                    # 2枚目以降は保存したreplayを使用
                    transformed = A.ReplayCompose.replay(replay, image=img)
                transformed_imgs.append(transformed["image"])

        # Depth Mapの変換
        transformed_depths = []
        for depth in depths:
            transformed_depth = self.depth_transform(image=depth)
            transformed_depths.append(transformed_depth["image"])

        # オリジナル画像をチャネル方向に結合 (C*3, H, W)
        img_tensor = torch.cat(transformed_imgs, dim=0)

        # Depth Mapをチャネル方向に結合 (3, H, W)
        depth_tensor = torch.cat(transformed_depths, dim=0)

        # 信号機マスクの生成
        mask = self.mask_generator.generate_masks(json_path)
        mask_tensor = torch.from_numpy(mask).permute(2, 0, 1)  # (8, H, W)

        # 画像, Depth Map, 信号機マスクをチャネル方向に結合 (C*3+3+8=20, H, W)
        combined_tensor = torch.cat([img_tensor, depth_tensor, mask_tensor], dim=0)

        # 数値特徴量の取得
        feature_tensor = torch.tensor(row[feature_cols].to_numpy(), dtype=torch.float32).squeeze(0)

        # ターゲットの準備
        if self.is_train:
            target = torch.tensor(row[self.target_cols].to_numpy(), dtype=torch.float32).squeeze(0)
            return combined_tensor, feature_tensor, target
        else:
            return combined_tensor, feature_tensor


### モデル

In [92]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=True):
        super().__init__()
        if p_trainable:
            self.p = nn.Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.pow(1.0 / self.p)
        return x


In [90]:
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=False, target_size=None, model_name=None):
        super().__init__()

        self.encoder = timm.create_model(
            cfg.model_name,
            pretrained=cfg.pretrained,
            num_classes=0,
            in_chans=cfg.in_chans,
            global_pool="",
        )

        self.n_features = self.encoder.num_features
        self.pool = GeM(p=3, eps=1e-6, p_trainable=True)
        self.n_numeric_features = 8  # 数値特徴量の次元数

        self.target_size = cfg.target_size if target_size is None else target_size

        self.fc = nn.Sequential(
            nn.Linear(self.n_features + self.n_numeric_features, 512),  # 画像特徴量と数値特徴量を結合
            nn.ReLU(),
            nn.Linear(512, self.target_size if target_size is None else target_size),
        )

    def forward(self, image, numeric_features):
        feature = self.encoder(image)
        feature = self.pool(feature).reshape(feature.size(0), -1)
        combined_features = torch.cat([feature, numeric_features], dim=1)
        output = self.fc(combined_features)
        return output


In [93]:
model = CustomModel(cfg.cnn)


### 学習

In [95]:
DEBUG = True

if DEBUG:
    cfg.cnn.num_epochs = 1
    train = train.head(100)


In [97]:
# 実験結果格納用のディレクトリを作成
japan_tz = pytz.timezone("Asia/Tokyo")
cfg.run_time = datetime.now(japan_tz).strftime("%Y%m%d_%H%M%S")
create_dir(cfg.data.results_path)

# CV用の配列を初期化
oof_predictions = np.zeros((len(train), len(cfg.target_cols)))
models = {}

for fold, (train_idx, valid_idx) in enumerate(gkf.split(train, groups=train["scene"])):
    print(f"Fold {fold + 1}")

    # データセットの作成
    train_dataset = CustomDataset(
        train[train_idx], cfg.data.img_root, cfg.data.json_root, cfg.data.depth_root, transform=get_train_transform()
    )
    valid_dataset = CustomDataset(
        train[valid_idx], cfg.data.img_root, cfg.data.json_root, cfg.data.depth_root, transform=get_valid_transform()
    )

    train_loader = DataLoader(train_dataset, batch_size=cfg.cnn.batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

    # モデル、損失関数、オプティマイザーの初期化
    model = CustomModel(cfg.cnn).to(device)
    criterion = nn.HuberLoss()
    # criterion = nn.L1Loss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.cnn.lr)
    total_steps = len(train_loader) * cfg.cnn.num_epochs
    scheduler = get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=total_steps * 0.1,
        num_training_steps=total_steps,
    )

    best_loss = float("inf")

    # 学習ループ
    for epoch in range(cfg.cnn.num_epochs):
        model.train()
        for images, numeric_features, targets in tqdm(train_loader):
            images = images.to(device)
            numeric_features = numeric_features.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(images, numeric_features)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()

        # 検証
        model.eval()
        valid_losses = []
        with torch.no_grad():
            for images, numeric_features, targets in valid_loader:
                images = images.to(device)
                numeric_features = numeric_features.to(device)
                targets = targets.to(device)
                outputs = model(images, numeric_features)
                loss = criterion(outputs, targets)
                valid_losses.append(loss.item())

        valid_loss = np.mean(valid_losses)
        print(f"Epoch {epoch + 1}, Valid Loss: {valid_loss:.4f}")

        # ベストモデルの保存
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), f"{cfg.data.results_path}/model_fold{fold}.pth")

    # ベストモデルでOOF予測
    model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
    model.eval()

    valid_dataset = CustomDataset(
        train[valid_idx], cfg.data.img_root, cfg.data.json_root, cfg.data.depth_root, transform=get_valid_transform()
    )
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

    with torch.no_grad():
        for i, (images, numeric_features) in enumerate(valid_loader):
            images = images.to(device)
            numeric_features = numeric_features.to(device)
            outputs = model(images, numeric_features)
            start_idx = i * cfg.cnn.batch_size
            end_idx = start_idx + outputs.shape[0]
            oof_predictions[valid_idx[start_idx:end_idx]] = outputs.cpu().numpy()

# CVスコアの計算（MAEの平均）
mae_scores = []
for i in range(len(cfg.target_cols)):
    mae = np.mean(np.abs(oof_predictions[:, i] - train[cfg.target_cols[i]].to_numpy()))
    mae_scores.append(mae)

cv_score = np.mean(mae_scores)
print(f"CV Score: {cv_score:.4f}")

# oofを保存
np.save(f"{cfg.data.results_path}/oof_predictions.npy", oof_predictions)


Directory created: ../../results/009/20241124_232225
Fold 1


  0%|          | 0/3 [00:00<?, ?it/s][ WARN:0@3081.763] global loadsave.cpp:241 findDecoder imread_('../../data/input/depth/000fb056f97572d384bae4f5fc1e0f28_20/image_t-1.0.png'): can't open/read file: check file path/integrity
[ WARN:0@3081.764] global loadsave.cpp:241 findDecoder imread_('../../data/input/depth/000fb056f97572d384bae4f5fc1e0f28_20/image_t-0.5.png'): can't open/read file: check file path/integrity
[ WARN:0@3081.764] global loadsave.cpp:241 findDecoder imread_('../../data/input/depth/000fb056f97572d384bae4f5fc1e0f28_20/image_t.png'): can't open/read file: check file path/integrity
  0%|          | 0/3 [00:00<?, ?it/s]


TypeError: image must be numpy array type

### 推論

In [12]:
# testの推論
test_dataset = CustomDataset(
    test, cfg.data.img_root, cfg.data.json_root, cfg.data.depth_root, transform=get_valid_transform(), is_train=False
)
test_loader = DataLoader(test_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

# fold分の予測値を格納する配列
test_predictions = np.zeros((len(test), len(cfg.target_cols), cfg.n_splits))

# 各foldのモデルで予測
for fold in range(cfg.n_splits):
    print(f"Predicting using fold {fold + 1} model")

    # モデルの読み込み
    model = CustomModel(cfg.cnn).to(device)
    model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
    model.eval()

    fold_predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = images.to(device)
            outputs = model(images)
            fold_predictions.append(outputs.cpu().numpy())

    # バッチごとの予測を結合
    fold_predictions = np.concatenate(fold_predictions, axis=0)
    test_predictions[:, :, fold] = fold_predictions

# fold分の予測値の平均を計算
final_predictions = test_predictions.mean(axis=2)

# submissionファイルの作成
exprs = [pl.Series(final_predictions[:, i]).alias(cfg.target_cols[i]) for i in range(len(cfg.target_cols))]
submission = sample_submission.with_columns(exprs)
submission.write_csv(f"{cfg.data.results_path}/submission.csv")
print("Submission file created!")

# 確認
submission.head()


Predicting using fold 1 model


/tmp/ipykernel_1568158/236869104.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
100%

Predicting using fold 2 model


100%|██████████| 54/54 [00:09<00:00,  5.44it/s]


Predicting using fold 3 model


100%|██████████| 54/54 [00:10<00:00,  5.05it/s]


Predicting using fold 4 model


100%|██████████| 54/54 [00:10<00:00,  5.14it/s]

Submission file created!


x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.910416,0.015323,-0.00203,4.111138,0.049189,0.001671,6.400317,0.093142,0.001914,8.764425,0.14443,0.008326,11.181863,0.203873,0.013329,13.635955,0.27344,0.019102
0.888904,0.273903,-0.002772,1.869887,0.726532,0.002171,2.899538,1.31719,0.011759,3.96669,2.022292,0.024279,5.108733,2.831406,0.041865,6.321622,3.753189,0.058737
2.079381,0.006609,-0.00427,4.220596,0.019128,-0.009004,6.19882,0.029642,-0.013304,8.010659,0.044004,-0.018825,9.672832,0.050245,-0.025318,11.221782,0.051523,-0.033896
0.916636,0.02805,-0.007229,1.805815,0.074985,-0.013208,2.588893,0.138594,-0.01687,3.264247,0.217395,-0.021215,3.85386,0.317263,-0.025356,4.371171,0.43165,-0.027858
1.283522,-0.002682,-0.005985,2.565668,-0.007541,-0.012687,3.715343,-0.017339,-0.020631,4.736634,-0.026279,-0.029341,5.637933,-0.035247,-0.039212,6.447492,-0.048434,-0.048612


In [13]:
# final_predictionsを保存
np.save(f"{cfg.data.results_path}/final_predictions.npy", final_predictions)


In [14]:
final_predictions.shape


(1727, 18)